In [18]:
import win32com.client
import pandas as pd
import os, glob
import os.path
import time
from datetime import datetime
from datetime import timedelta

instCpCybos = win32com.client.Dispatch('CpUtil.CpCybos')
instCpStockCode = win32com.client.Dispatch('CpUtil.CpStockCode')
instCpCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
instStockChart = win32com.client.Dispatch('CpSysDib.StockChart')

In [5]:
# 크레온 API 연결 확인
# <리턴값 종류 확인 필요>
def check_connection():
    if instCpCybos.IsConnect == 1:
        print('정상적으로 연결')
    else:
        print('연결 실패')
        
check_connection()

정상적으로 연결


In [6]:
# 종목 이름으로 찾기
def search_by_name(stock_name):
    count = instCpStockCode.GetCount()
    for i in range(count):
        if instCpStockCode.GetData(1,i) == stock_name:
            # GetData(x,y)
            # x : 0 => 종목 코드
            # x : 1 => 종목명
            # x : 2 => ISIN 코드(ISIN : 국제 증권 식별 번호)
            return [instCpStockCode.GetData(0,i), instCpStockCode.GetData(1,i)]

search_by_name('삼성전자')

['A005930', '삼성전자']

In [7]:
# 특정 코드의 종목 찾기
def search_by_code(stock_code):
    count = instCpStockCode.GetCount()
    for i in range(count):
        if instCpStockCode.GetData(0,i) == stock_code:
            # GetData(x,y)
            # x : 0 => 종목 코드
            # x : 1 => 종목명
            # x : 2 => ISIN 코드(ISIN : 국제 증권 식별 번호)
            return [instCpStockCode.GetData(0,i), instCpStockCode.GetData(1,i)]

search_by_code('A005930')

['A005930', '삼성전자']

In [8]:
# 종목명 전체 리스트 가져오기
# <선물이랑 ETF 쪽은 구분 필요>
def get_stock_list():
    count = instCpStockCode.GetCount()
    stock_list = []
    
    for i in range(count):
        stock_list.append(instCpStockCode.GetData(1,i))
    
    return stock_list

# get_stock_list()

In [9]:
# print(instCpCodeMgr.GetStockListByMarket(1))

In [10]:
# 전체 종목 가져오기
codelist = instCpCodeMgr.GetStockListByMarket(1)
kospi = {}
for code in codelist:
    name = instCpCodeMgr.CodeToName(code)
    kospi[code] = name
    
# kospi

In [11]:
# 
for i, code in enumerate(codelist):
    secondCode = instCpCodeMgr.GetStockSectionKind(code)
    name = instCpCodeMgr.CodeToName(code)
    # print(i, code, secondCode, name)

In [ ]:
# 
code_list = []
for i, code in enumerate(codelist):
    secondCode = instCpCodeMgr.GetStockSectionKind(code)
    name = instCpCodeMgr.CodeToName(code)
    code_list.append(code)
# print(code_list)


In [55]:
import time
# 특정 범위 일자 종목 데이터 가져오기
def get_stock_info(stock_code, start_day, end_day, type):
    
    instStockChart.SetInputValue(0, stock_code) # 종목명
    instStockChart.SetInputValue(1, ord('1')) # 1 : 기간으로 요청, 2: 개수로 요청
    instStockChart.SetInputValue(3, start_day) # 요청 시작일
    instStockChart.SetInputValue(2, end_day) # 요청 종료일
    # instStockChart.SetInputValue(4, request_num) # 요청 개수
    '''
    # 요청할 데이터 종류(리스트 형태로 요청 가능)
    0 : 날짜, 1 : 시간 - hhmm, 2 : 시가, 3 : 고가, 4 : 저가, 5 : 종가
    6 : 전일대비, 8 : 거래량, 9 : 거래대금, 13 : 시가총액
    '''
    instStockChart.SetInputValue(5, [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, 15, 16, 17,18, 19,20,21,22,23,24,25,26])
    instStockChart.SetInputValue(6, ord(type)) # 'D' : 일봉, 'm' : 분봉
    instStockChart.SetInputValue(9, ord('1'))

    instStockChart.BlockRequest() # 위 정보로 요청

    numrow, numcolumn = instStockChart.GetHeaderValue(3), instStockChart.GetHeaderValue(2)

    index = []
    for i in range(numrow):
        index_ = str(instStockChart.GetDataValue(0,i))
        index.append(index_)

    stock_info = pd.DataFrame(columns=numcolumn[1:], index=index)

    for num in range(numrow):
        for col in range(len(numcolumn)):
            # 1,2,3,4,5,6,7,8,9, 10
            stock_info.iloc[num, col-1] = str(instStockChart.GetDataValue(col,num))
    
    return stock_info

donghwa_m = get_stock_info('A005930', '20221029', '20221102', 'm')
donghwa_m

,시간,시가,고가,저가,종가,...,등락주선,등락비율,예탁금,주식회전율,거래성립률
20221102,1530,59600,59600,59600,59600,...,0,0.0,0,0.0,0.0
20221102,1520,59500,59600,59500,59500,...,0,0.0,0,0.0,0.0
20221102,1519,59500,59600,59500,59600,...,0,0.0,0,0.0,0.0
20221102,1518,59500,59600,59500,59600,...,0,0.0,0,0.0,0.0
20221102,1517,59500,59600,59500,59600,...,0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
20221031,1449,59600,59600,59500,59500,...,0,0.0,0,0.0,0.0
20221031,1448,59600,59600,59500,59600,...,0,0.0,0,0.0,0.0
20221031,1447,59600,59600,59500,59600,...,0,0.0,0,0.0,0.0
20221031,1446,59500,59600,59500,59500,...,0,0.0,0,0.0,0.0


In [53]:
pd.options.display.max_columns=10
donghwa_d = get_stock_info('A005930', '20201023', '20221102', 'D')
donghwa_d

,시간,시가,고가,저가,종가,...,등락주선,등락비율,예탁금,주식회전율,거래성립률
20221102,0,59700,60000,59300,59600,...,0,0.0,0,0.0,0.0
20221101,0,59900,60300,59500,60000,...,0,0.0,0,0.0,0.0
20221031,0,58100,59900,58000,59400,...,0,0.0,0,0.0,0.0
20221028,0,58900,59200,57200,57300,...,0,0.0,0,0.0,0.0
20221027,0,59700,60100,58900,59500,...,0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
20201029,0,58200,58500,57500,58100,...,0,0.0,0,0.0,0.0
20201028,0,59400,59400,58800,59000,...,0,0.0,0,0.0,0.0
20201027,0,60300,60300,59500,59800,...,0,0.0,0,0.0,0.0
20201026,0,60400,61100,60100,60400,...,0,0.0,0,0.0,0.0


In [47]:
donghwa_m.index

Index(['20221102', '20221102', '20221102', '20221102', '20221102', '20221102',
       '20221102', '20221102', '20221102', '20221102',
       ...
       '20221031', '20221031', '20221031', '20221031', '20221031', '20221031',
       '20221031', '20221031', '20221031', '20221031'],
      dtype='object', length=799)

In [54]:
value_list = ['전일대비','상장주식수','시가총액','외국인주문한도수량','외국인주문가능수량','외국인현보유수량','외국인현보유비율','수정주가일자','수정주가비율','기관순매수량','기관누적순매수량']
for day in donghwa_m.index:
    donghwa_m.loc[day, value_list] = donghwa_d.loc[day, value_list].values
# donghwa_d.loc['20221102', ['전일대비']].values
donghwa_m

,시간,시가,고가,저가,종가,...,등락주선,등락비율,예탁금,주식회전율,거래성립률
20221102,1530,59600,59600,59600,59600,...,0,0.0,0,0.0,0.0
20221102,1520,59500,59600,59500,59500,...,0,0.0,0,0.0,0.0
20221102,1519,59500,59600,59500,59600,...,0,0.0,0,0.0,0.0
20221102,1518,59500,59600,59500,59600,...,0,0.0,0,0.0,0.0
20221102,1517,59500,59600,59500,59600,...,0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
20221031,1449,59600,59600,59500,59500,...,0,0.0,0,0.0,0.0
20221031,1448,59600,59600,59500,59600,...,0,0.0,0,0.0,0.0
20221031,1447,59600,59600,59500,59600,...,0,0.0,0,0.0,0.0
20221031,1446,59500,59600,59500,59500,...,0,0.0,0,0.0,0.0


In [14]:
import time
# 전체 일자 분봉 종목 데이터 가져오기
def get_stock_info_auto(stock_code, end_day, type):
    
    day_format = '%Y%m%d'
    minus_day = timedelta(days=5)
    
    transfer_end_day = datetime.strptime(end_day, day_format) # end_day 변환
    pre_day = datetime.strftime(transfer_end_day - minus_day, day_format) # pre_day 계산
    
    stock_df = get_stock_info(stock_code, pre_day, end_day, type) # 첫번째 DF 생성

    for day in range(150): # range 범위 수정하지 말 것
        time.sleep(0.25)
        dayily_df = pd.DataFrame()
        transfer_pre_day = datetime.strptime(pre_day, day_format)
        next_pre_day = datetime.strftime(transfer_pre_day - minus_day, day_format)

        info_second = get_stock_info(stock_code, next_pre_day, pre_day, type)
        
        pre_day = next_pre_day
        
        stock_df = pd.concat([stock_df, info_second], axis=0)
    
    stock_name = search_by_code(stock_code)
    print(stock_name, '남은 요청 횟수 :{0}'.format(instCpCybos.GetLimitRemainCount(0)))
    stock_df.to_csv('../data/{0}_{1}.csv'.format(stock_name[0][1:],stock_name[1]), encoding='utf-8-sig')

    return stock_df

# samsung = get_stock_info_auto('A005930', '20221101', 'm')
# samsung

In [ ]:
FOLDER_PATH = "../data/*"
file_list = glob.glob(FOLDER_PATH)
print("파일 개수 ", len(file_list))

for code in code_list[len(file_list):]:
    get_stock_info_auto(code, '20221101', 'm')
